# Backtest Misc

Examples of backtesting for 
* multiple series 
* multiple models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from orbit.lgt import LGT
from orbit.dlt import DLT
from orbit.backtest.backtest import TimeSeriesSplitter, Backtest
from orbit.utils.metrics import mape, smape, wmape
from orbit.backtest.functions import run_multi_series_backtest

ModuleNotFoundError: No module named 'orbit.lgt'

## Load data

In [ ]:
data_path = "../examples/data/m4_weekly.csv"
data = pd.read_csv(data_path, parse_dates=['date'])

In [ ]:
print("Total keys: {}".format(len(data['key'].unique())))
data.head(3)

## Settings

In [ ]:
# data settings
date_col = "date"
response_col = "value"
key_col = "key"
seasonality = 52

# backtest settings
min_train_len = 120
forecast_len = 13
incremental_len = 5
n_splits = 1
window_type = "expanding"
seed = 2019

In [ ]:
lgt_map = LGT(
    response_col=response_col,
    date_col=date_col,
    seasonality=seasonality,
    seed=seed,
    infer_method='map',
    predict_method='map',
    is_multiplicative=True
)

dlt_map = DLT(
    response_col=response_col,
    date_col=date_col,
    seasonality=seasonality,
    seed=seed,
    infer_method='map',
    predict_method='map',
)


In [ ]:
bt_models = {
    'LGT-MAP': lgt_map,
    'DLT-MAP': dlt_map
}

## Backtest Multiple Series

In [ ]:
bt_result, bt_scores = run_multi_series_backtest(
    data=data, 
    response_col=response_col, 
    key_col=key_col, 
    date_col=date_col,
    model=lgt_map,
    min_train_len=min_train_len, 
    incremental_len=incremental_len, 
    forecast_len=forecast_len, 
    predicted_col='prediction', 
    n_splits=n_splits,
    window_type = window_type
)

In [ ]:
bt_scores.head()

## Backtest Multiple Models (and Multiple Series)

In [ ]:
bt_result_list = []
bt_scores_list = []
for mod_name, mod in bt_models.items():
    bt_result, bt_scores = run_multi_series_backtest(
        data=data, 
        response_col=response_col, 
        key_col=key_col, 
        date_col=date_col,
        model=lgt_map,
        min_train_len=min_train_len, 
        incremental_len=incremental_len, 
        forecast_len=forecast_len, 
        predicted_col='prediction', 
        n_splits=n_splits,
        window_type = window_type
    )
    bt_result['model'] = mod_name
    bt_scores['model'] = mod_name
    bt_result_list.append(bt_result)
    bt_scores_list.append(bt_scores)

In [ ]:
bt_result = pd.concat(bt_result_list, axis=0)
bt_scores = pd.concat(bt_scores_list, axis=0)

In [ ]:
bt_scores.head()